In [1]:
import glob
import json
import pandas as pd

In [2]:
import sys
import os
# Access to the utils directory
current_dir = os.getcwd()
utils_dir = os.path.join(current_dir, '..', 'utils')
sys.path.append(utils_dir)
from string_utils import TableValuesPostTreatment, NormalizeText

[nltk_data] Downloading package punkt to /home/STual/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/STual/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1. Init Gold Standard jsons

In [3]:
PAGE_CSVS = sorted(glob.glob("/home/STual/DAN-cadastre/data/LHAY/corrected_updated/1*.csv"))
ENTITIES_CSV = sorted(glob.glob("/home/STual/DAN-cadastre/data/LHAY/corrected_updated/*_entities.csv"))
SAVE_FOLDER = "/home/STual/DAN-cadastre/data/LHAY/corrected_jsons"

In [4]:
len(PAGE_CSVS)

26

In [5]:
def list_in_cell(key,sep="|",join_sep = ", "):

    if pd.notna(row[key]) and sep in row[key]:
        return join_sep.join(row[key].split(sep)),
    elif pd.notna(row[key]) and "|" not in row[key]:
        return [row[key]]
    else:
        return []

In [6]:
for PAGE in PAGE_CSVS:
    df_ = pd.read_csv(PAGE,encoding='utf8')
    df = df_[df_["line_status"].isin(["VALIDE", "MANQUANTE"])]
    page_json = {"text":"","confidences":{"total":1.0},"language_model":"","objects":[],"entities":[]}
    page_uuid = df.iloc[0]["page_uuid"]
    page_text = ""
    for _, row in df.iterrows():
        if row["line_uuid"] != "28215bed-0304-4dd6-8360-2fd0f81f2b43":
            line_ent = {}
            line = ""
            if pd.notna(row["plot_address"]) and row["plot_address"] != "":
                line_ent["Ⓓ"] = {}
                line_ent["Ⓓ"]["text"] = row["plot_address"]
                line_ent["Ⓓ"]["offset"] = "NC"
                line_ent["Ⓓ"]["length"] = "NC"
                line_ent["Ⓓ"]["interpreted_text"] = ""
                line_ent["Ⓓ"]["uris"] = "http://rdf.geohistoricaldata.org/id/landmark/" + row["landmark_uuid"]
                line += "Ⓓ" + row["plot_address"] + " "
                
            if pd.notna(row["taxpayer_num"]) and row["taxpayer_num"] != "":
                line_ent["Ⓖ"] = {}
                if isinstance(row["taxpayer_num"],float):
                    line_ent["Ⓖ"]["text"] = str(int(row["taxpayer_num"]))
                elif isinstance(row["taxpayer_num"],int):
                    line_ent["Ⓖ"]["text"] = str(row["taxpayer_num"])
                line_ent["Ⓖ"]["offset"] = "NC"
                line_ent["Ⓖ"]["length"] = "NC"
                line_ent["Ⓖ"]["interpreted_text"] = ""
                line += "Ⓖ" + str(row["taxpayer_num"]) + " "
    
            if pd.notna(row["taxpayer_cell"]) and row["taxpayer_cell"] != "":
                line_ent["Ⓒ"] = {}
                line_ent["Ⓒ"]["text"] = row["taxpayer_cell"]
                line_ent["Ⓒ"]["offset"] = "NC"
                line_ent["Ⓒ"]["length"] = "NC"
                line_ent["Ⓒ"]["interpreted_text"] = row["taxpayer_interpreted"]
                line_ent["Ⓒ"]["uris"] = ["http://rdf.geohistoricaldata.org/id/taxpayer/" + row["taxpayer_uuid"]]
                line += "Ⓒ" + row["taxpayer_cell"] + " "
                line_ent["Ⓒ"]["ner_interpreted_text_grouped"] = {}
                
                if pd.notna(row["name"]):
                    line_ent["Ⓒ"]["ner_interpreted_text_grouped"]["name"] = row["name"]
                else:
                    line_ent["Ⓒ"]["ner_interpreted_text_grouped"]["name"] = ""
    
                if pd.notna(row["firstnames"]):
                    if '|' in row["firstnames"]:
                        ls = row["firstnames"].split('|')
                        name = ' et '.join(ls)
                    else:
                        name = row["firstnames"]
                    line_ent["Ⓒ"]["ner_interpreted_text_grouped"]["firstnames"] = name
                else:
                    line_ent["Ⓒ"]["ner_interpreted_text_grouped"]["firstnames"] = ""
    
                if pd.notna(row["birthname"]):
                    if '|' in row["birthname"]:
                        ls = row["birthname"].split('|')
                        bname = ' et '.join(ls)
                    else:
                        bname = row["birthname"]
                    line_ent["Ⓒ"]["ner_interpreted_text_grouped"]["birthname"] = bname
                else:
                    line_ent["Ⓒ"]["ner_interpreted_text_grouped"]["birthname"] = ""
                    
                line_ent["Ⓒ"]["ner_interpreted_text_grouped"]["address"] = list_in_cell("addresses",sep="|",join_sep = ", ")
                    
                line_ent["Ⓒ"]["ner_interpreted_text_grouped"]["familystatus"] = list_in_cell("familystatus",sep="|",join_sep = ", ")
    
                line_ent["Ⓒ"]["ner_interpreted_text_grouped"]["activity"] = list_in_cell("activities",sep="|",join_sep = ", ")
                    
                line_ent["Ⓒ"]["ner_interpreted_text_grouped"]["title"] = list_in_cell("titles",sep="|",join_sep = ", ")
                     
            if pd.notna(row["plot_number"]) and row["plot_number"] != "":
                line_ent["Ⓕ"] = {}
                line_ent["Ⓕ"]["text"] = row["plot_number"]
                line_ent["Ⓕ"]["offset"] = "NC"
                line_ent["Ⓕ"]["length"] = "NC"
                if isinstance(row["plot_number"],str):
                    line_ent["Ⓕ"]["interpreted_text"] = row["plot_number"]
                    line_ent["Ⓕ"]["postcorrected_text"] = row["plot_number"].replace('er','').replace('↓','').replace('↑','').replace('→','')
                elif isinstance(row["plot_number"],float):
                    line_ent["Ⓕ"]["interpreted_text"] = str(int(row["plot_number"]))
                    line_ent["Ⓕ"]["postcorrected_text"] = str(int(row["plot_number"]))
                elif isinstance(row["plot_number"],int):
                    line_ent["Ⓕ"]["interpreted_text"] = str(row["plot_number"])
                    line_ent["Ⓕ"]["postcorrected_text"] = str(row["plot_number"])
                line += "Ⓕ" + str(row["plot_number"]) + " "
    
            if pd.notna(row["plotnature"]) and row["plotnature"] != "":
                line_ent["Ⓔ"] = {}
                line_ent["Ⓔ"]["text"] = row["plotnature"]
                line_ent["Ⓔ"]["offset"] = "NC"
                line_ent["Ⓔ"]["length"] = "NC"
                line_ent["Ⓔ"]["interpreted_text"] = row["plotnature"]
                if pd.notna(row["skosNature"]):
                    line_ent["Ⓔ"]["uris"] = ["http://rdf.geohistoricaldata.org/id/codes/cadastre/plotNature/" + row["skosNature"]]
                line += "Ⓔ" + row["plotnature"] + " "
                
            line_ent["uri"] = row["line_uuid"]
            line_ent["status"] = row["line_status"]
            line_ent["detection_order"] = int(row["line_detection_order"])
            line_ent["true_order"] = int(row["line_true_order"])
            page_text = page_text + '\n' + line 
            page_json["entities"].append(line_ent)
            page_json["objects"].append({"confidence":"NC","polygon":"NC","text":line,"text_confidence":1.0})

    page_json["text"] = page_text
    with open(SAVE_FOLDER + '/' + page_uuid + '.json', "w", encoding="utf-8") as f:
        json.dump(page_json, f, ensure_ascii=False, indent=4)

In [7]:
df

,page_uuid,line_uuid,line_detection_order,line_true_order,line_status,taxpayer_num,taxpayer_cell,taxpayer_interpreted,name,firstnames,...,addresses,titles,familystatus,plot_number,plot_address,plotnature,comment,skosNature,taxpayer_uuid,landmark_uuid
0,9e52f7f9-0e66-4753-bcf3-d3cb0916678c,b66497f6-5c2f-40c6-800b-7c37410ec850,0,0.0,VALIDE,136,Grenier louis,Grenier louis,Grenier,louis,...,NaN,NaN,NaN,346,La plaine→de L'Hay,Terre,NaN,Terre,7a040ad6-4967-4b63-8b1a-f3cc6f822d66,65dac2d2-8049-4cdd-88c4-cf218b980d6d
1,9e52f7f9-0e66-4753-bcf3-d3cb0916678c,c19d661a-a57e-4495-9faf-fcff26014a62,1,1.0,VALIDE,130,Godefroy antoine,Godefroy antoine,Godefroy,antoine,...,NaN,NaN,NaN,347,$,T,NaN,Terre,06ee3332-3587-4e7e-b189-76bc0109f4d8,65dac2d2-8049-4cdd-88c4-cf218b980d6d
2,9e52f7f9-0e66-4753-bcf3-d3cb0916678c,8db372ae-3c6d-443b-b258-8ecc80169d48,2,2.0,VALIDE,107,Devernouailles à Vitry,Devernouailles à Vitry,Devernouailles,NaN,...,Vitry,NaN,NaN,348,§,T,NaN,Terre,200996d7-4b0c-4db1-bace-d33b121c4e87,65dac2d2-8049-4cdd-88c4-cf218b980d6d
3,9e52f7f9-0e66-4753-bcf3-d3cb0916678c,1470289a-33eb-4fe2-9139-c56e96a69bc1,3,3.0,VALIDE,46,Bourdillat j↑h↓,Bourdillat j↑h↓,Bourdillat,j↑h↓,...,NaN,NaN,NaN,349,§,T,NaN,Terre,d98d3ac9-18b2-4de7-9a5c-50642d46913f,65dac2d2-8049-4cdd-88c4-cf218b980d6d
4,9e52f7f9-0e66-4753-bcf3-d3cb0916678c,7965414a-205c-4c03-80af-7a1d767745a2,4,4.0,VALIDE,301,Thibault jean claude,Thibault jean claude,Thibault,jean claude,...,NaN,NaN,NaN,350,§,T.pl.,NaN,TerrePlantee,834eb74f-007f-4952-bbf6-cd1edbb0698b,65dac2d2-8049-4cdd-88c4-cf218b980d6d
5,9e52f7f9-0e66-4753-bcf3-d3cb0916678c,e41a3858-d596-41c1-8e5a-869972f42618,5,5.0,VALIDE,74,Chevalier claude ×pe± ↑fils↓,Chevalier claude↑fils↓,Chevalier,claude,...,NaN,NaN,fils,351,§,T.pl.,NaN,TerrePlantee,62e15430-dc04-4faf-8091-f03c4825357a,65dac2d2-8049-4cdd-88c4-cf218b980d6d
6,9e52f7f9-0e66-4753-bcf3-d3cb0916678c,c7578150-677e-4a4b-b567-6630b66b0ce9,6,6.0,VALIDE,284,Serre de st Roman,Serre de st Roman,Serre de st Roman,NaN,...,NaN,NaN,NaN,352,§,T,NaN,Terre,34fcbbb5-159c-4604-89af-92f9baca0423,65dac2d2-8049-4cdd-88c4-cf218b980d6d
7,9e52f7f9-0e66-4753-bcf3-d3cb0916678c,bf68c90d-ca82-4510-b773-a2c16be89131,13,7.0,MANQUANTE,74,Chevalier claude fils,Chevalier claude fils,Chevalier,claude,...,NaN,NaN,fils,352↑bis↓,§,☼,NaN,Terre,62e15430-dc04-4faf-8091-f03c4825357a,65dac2d2-8049-4cdd-88c4-cf218b980d6d
8,9e52f7f9-0e66-4753-bcf3-d3cb0916678c,4aa569c3-7a90-41b2-8971-166533441fc8,14,8.0,MANQUANTE,71,Chevalier ×f↑ois↓ pe± → angélique↑fille↓,Chevalier ×f↑ois↓ pe±→angélique↑fille↓,Chevalier,angélique,...,NaN,NaN,fille,353,§,T.pl.,NaN,TerrePlantee,46f36867-07ef-4387-b924-cfdb14003f96,65dac2d2-8049-4cdd-88c4-cf218b980d6d
9,9e52f7f9-0e66-4753-bcf3-d3cb0916678c,15b020a5-b966-4068-9812-f0d42f394b64,8,9.0,VALIDE,74,Chevalier claude fils,Chevalier claude fils,Chevalier,claude,...,NaN,NaN,fils,353↑bis↓,§,T.pl.,NaN,TerrePlantee,62e15430-dc04-4faf-8091-f03c4825357a,65dac2d2-8049-4cdd-88c4-cf218b980d6d


## 2. Post-treat data
* idem
* cells to merge

In [8]:
from string_utils import create_split_dictionary
import uuid

chars_to_split = ["Ⓐ","Ⓑ","Ⓒ","Ⓓ","Ⓔ","Ⓕ","Ⓖ"]
ROOT = "/home/STual/DAN-cadastre/"
JSONS = glob.glob("/home/STual/DAN-cadastre/data/LHAY/corrected_jsons/*.json")

for JSON in JSONS:
    with open(JSON,'r',encoding='utf8') as f:
        page = json.load(f)
    page_uuid = JSON.replace(ROOT,"").replace('.json','')

    #Deal with items and ditto
    page["entities"] = TableValuesPostTreatment.process_idem_ditto_replacements(page["entities"], SPECIAL_VALUE="MISSING", 
                                    idem_list=['§', 'Ø', 'id', 'idem', 'le meme', 'la meme', 'les memes'],
                                    ditto_list=['☼'],to_remove=["uri","detection_order","true_order","status"])
    
    json_path = os.path.join(ROOT, page_uuid + '.json')
    
    with open(json_path,'w', encoding='utf-8') as f:
        json.dump(page, f, ensure_ascii=True, indent=4)